In [16]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [39]:
characters = [c for c in 'abcdefghijklmnopqrstuvwxyz']

In [40]:
word_dict = {n: i for i, n in enumerate(characters)}
number_dict = {i: w for i, w in enumerate(characters)}
n_class = len(word_dict)

In [65]:
data = ["kind", "yale", "make", "spin", "tale", "wire", "book", "case", "love", "like", "bore"]

In [66]:
def make_batch(data):
    input_batch, target_batch = [], []

    for seq in data:
        input = [word_dict[n] for n in seq[:-1]]
        target = word_dict[seq[-1]] 
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return Variable(torch.Tensor(input_batch)), Variable(torch.LongTensor(target_batch))

In [67]:
dtype = torch.FloatTensor

In [68]:
n_hidden = 100

In [69]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden)
        self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype))

    def forward(self, X):
        input = X.transpose(0, 1) 

        # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        hidden_state = Variable(torch.zeros(1, len(X), n_hidden))   
        
        # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        cell_state = Variable(torch.zeros(1, len(X), n_hidden))     

        outputs, (h_n, c_n) = self.lstm(input, (hidden_state, cell_state))
        outputs = outputs[-1]  
        model = torch.mm(outputs, self.W) + self.b  
        return model

In [70]:
input_batch, target_batch = make_batch(data)

In [71]:
model = Network()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [72]:
epochs = 500
for epoch in range(1, epochs+1):
    optimizer.zero_grad()

    output = model(input_batch)
    loss = criterion(output, target_batch)
    if epoch % 100 == 0:
        print(f"epoch: {epoch} ===> loss: {loss}")

    loss.backward()
    optimizer.step()

epoch: 100 ===> loss: 0.024077434092760086
epoch: 200 ===> loss: 0.0050083971582353115
epoch: 300 ===> loss: 0.002119732555001974
epoch: 400 ===> loss: 0.0011644053738564253
epoch: 500 ===> loss: 0.0007351472740992904


In [79]:
f_in = []
for word in test:
    f_in.append(word[:-1])

In [80]:
f_in

['are', 'bat', 'bir', 'bes', 'cal', 'cod', 'dow']

In [76]:
f_out = []
for index in predict:
    f_out.append(number_dict[index.item()])

In [77]:
for i in range(len(f_in)):
    print(f_in[i]," ==> ", f_out[i])

kin  ==>  d
yal  ==>  e
mak  ==>  e
spi  ==>  n
tal  ==>  e
wir  ==>  e
boo  ==>  k
cas  ==>  e
lov  ==>  e
lik  ==>  e
bor  ==>  e
